In [346]:
import pandas as pd
import numpy as np

In [347]:
df = pd.read_csv('All orders - short custom description.csv', low_memory=False)
df.drop_duplicates(inplace=True)
df.drop(columns=['Category'], inplace=True)
df['customs_description'] = df['customs_description'].astype(str)
df['F_Cat'] = df['F_Cat'].astype(str)
df['quantity'] = df['quantity'].astype(int)
temp_df = df['attribute_summary'].str.split(': ', expand=True)
temp_df.columns = ['F_Size', 'Size']
df['attribute_summary'] = temp_df['Size']

df.rename(columns={'attribute_summary': 'Size', 'model': 'Model', 'customs_description': 'Category'}, inplace=True)

In [348]:
import matplotlib
list_colours = []
for name, hex in matplotlib.colors.cnames.items():
    name = name.title()
    list_colours.append(name)

# list_colours = []
colours_df = pd.read_csv('colours.csv')
colours_df['colours'] = colours_df['colours'].str.strip()
for j in colours_df['colours'].unique().tolist():
    list_colours.append(j)

for i in list_colours:
    i = i.title()

list_colours = list(dict.fromkeys(list_colours))

df['Colour'] = df['title'].str.findall(r'(?i)\b(?:{})\b'.format('|'.join(list_colours))).apply(', '.join)

df2 = df['Colour'].str.split(',', expand=True)
df['Colour'] = df2[0]

In [349]:
orig_df = df.copy()

In [350]:
d1, d2 = '2024-02-01', '2024-02-29'
month_abb = 'Feb'
channel = 'Debenhams'

In [351]:
df = df[(df['date'] >= d1) & (df['date'] <= d2)]
df = df[df['Channel'] == channel]

In [352]:
# # temp_show_df = df[(df['Category'] == 'Ladies > Leather Jackets')]
# df.groupby(["order_id", "title"]).filter(lambda x: len(x['order_state'].unique()) > 1)[['order_date', 'order_id', 'order_state', 'title', 'Size']]

In [353]:
df = df[(df['Category'] == 'Ladies > Leather Jackets')]
df = df[df['Model'] != '5 pads set']

df['Model'] = df['Model'].fillna("(No assigned category)")
df['Colour'] = df['Colour'].fillna("(No assigned colour)")
df['Size'] = df['Size'].fillna("(No assigned size)")

if channel == 'Leather Company':
    temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_dispatched_2 = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_refunded = df[df['order_state'] == 'Order Refunded']
    temp_refunded_2 = df[df['order_state'] == 'Order Refunded']

elif channel == 'Debenhams':
    temp_fix_1_df = df.groupby(["order_id", "title"]).filter(lambda x: len(x['order_state'].unique()) == 1)
    temp_fix_2_df = df.groupby(["order_id", "title"]).filter(lambda x: len(x['order_state'].unique()) > 1)
    temp_fix_2_df = temp_fix_2_df[temp_fix_2_df['order_state'] == 'Order Dispatched']
    df = pd.concat([temp_fix_1_df, temp_fix_2_df], ignore_index=True)
    temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_dispatched_2 = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_refunded = df[df['order_state'] == 'Order Refunded']
    temp_refunded_2 = df[df['order_state'] == 'Order Refunded']

temp_dispatched = temp_dispatched[['Category', 'Model', 'Colour', 'Size', 'quantity']]
temp_dispatched = temp_dispatched.groupby(['Category', 'Model', 'Colour', 'Size'])['quantity'].sum()
temp_dispatched = temp_dispatched.unstack(level=['Colour', 'Size'])
temp_dispatched = temp_dispatched.assign(Total=temp_dispatched.sum(1, min_count=2))
temp_dispatched = temp_dispatched.stack(level=['Colour', 'Size'], sort=False)
# temp_dispatched = temp_dispatched.unstack(level=['Size'])
# temp_dispatched = temp_dispatched.assign(Total=temp_dispatched.sum(1, min_count=2))
# temp_dispatched = temp_dispatched.stack(level=['Size'])
temp_dispatched = temp_dispatched.to_frame(name='quantity')
temp_dispatched.rename(columns={'quantity': 'Units Sold'}, inplace=True)

temp_dispatched_2 = temp_dispatched_2[['Category', 'Model', 'Colour', 'Size', 'price_inc']]
temp_dispatched_2 = temp_dispatched_2.groupby(['Category', 'Model', 'Colour', 'Size'])['price_inc'].sum()
temp_dispatched_2 = temp_dispatched_2.unstack(level=['Colour', 'Size'])
temp_dispatched_2 = temp_dispatched_2.assign(Total=temp_dispatched_2.sum(1, min_count=2))
temp_dispatched_2 = temp_dispatched_2.stack(level=['Colour', 'Size'], sort=False)
# temp_dispatched_2 = temp_dispatched_2.unstack(level=['Size'])
# temp_dispatched_2 = temp_dispatched_2.assign(Total=temp_dispatched_2.sum(1, min_count=2))
# temp_dispatched_2 = temp_dispatched_2.stack(level=['Size'], sort=False)
temp_dispatched_2 = temp_dispatched_2.to_frame(name='price_inc')
temp_dispatched_2.rename(columns={'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_dispatched = pd.concat([temp_dispatched, temp_dispatched_2], axis=1)



temp_refunded = temp_refunded[['Category', 'Model', 'Colour', 'Size', 'quantity']]
temp_refunded = temp_refunded.groupby(['Category', 'Model', 'Colour', 'Size'])['quantity'].sum()
temp_refunded = temp_refunded.unstack(level=['Colour', 'Size'])
temp_refunded = temp_refunded.assign(Total=temp_refunded.sum(1, min_count=2))
temp_refunded = temp_refunded.stack(level=['Colour', 'Size'], sort=False)
# temp_refunded = temp_refunded.unstack(level=['Size'])
# temp_refunded = temp_refunded.assign(Total=temp_refunded.sum(1, min_count=2))
# temp_refunded = temp_refunded.stack(level=['Size'])
temp_refunded = temp_refunded.to_frame(name='quantity')
temp_refunded.rename(columns={'quantity': 'Units Refunded'}, inplace=True)

temp_refunded_2 = temp_refunded_2[['Category', 'Model', 'Colour', 'Size', 'price_inc']]
temp_refunded_2 = temp_refunded_2.groupby(['Category', 'Model', 'Colour', 'Size'])['price_inc'].sum()
temp_refunded_2 = temp_refunded_2.unstack(level=['Colour', 'Size'])
temp_refunded_2 = temp_refunded_2.assign(Total=temp_refunded_2.sum(1, min_count=2))
temp_refunded_2 = temp_refunded_2.stack(level=['Colour', 'Size'], sort=False)
# temp_refunded_2 = temp_refunded_2.unstack(level=['Size'])
# temp_refunded_2 = temp_refunded_2.assign(Total=temp_refunded_2.sum(1, min_count=2))
# temp_refunded_2 = temp_refunded_2.stack(level=['Size'])
temp_refunded_2 = temp_refunded_2.to_frame(name='price_inc')
temp_refunded_2.rename(columns={'price_inc': 'Total Refund (£)'}, inplace=True)

temp_refunded = pd.concat([temp_refunded, temp_refunded_2], axis=1)

temp_ladies = pd.concat([temp_dispatched, temp_refunded], axis=1)

temp_ladies = temp_ladies[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp_ladies['Units Sold'] = temp_ladies['Units Sold'].fillna(0)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].fillna(0)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].fillna(0)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].fillna(0)

df_total_sold = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
df_total_refund = df[df['order_state'] == 'Order Refunded']

total_list = [df_total_sold['quantity'].sum(), df_total_sold['price_inc'].sum(),
            df_total_refund['quantity'].sum(), df_total_refund['price_inc'].sum()]
temp_ladies.loc['Total', :] = total_list

temp_ladies['Final Revenue (£)'] = temp_ladies['Total Revenue (£)'] - temp_ladies['Total Refund (£)']

temp_ladies['Units Sold'] = temp_ladies['Units Sold'].astype(int)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].astype(int)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].astype(int)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].astype(int)
temp_ladies['Final Revenue (£)'] = temp_ladies['Final Revenue (£)'].astype(int)

In [354]:
temp_ladies

Units Sold  \
Category                 Model      Colour   Size               
Ladies > Leather Jackets 2810       Black    14             3   
                                             16             1   
                                             18             2   
                                    Coffee   10             2   
                                             12             1   
                                    Total                   9   
                         AWL-281    Black    14             1   
                                             18             1   
                                    Burgandy 22             1   
                                    Grey     12             1   
                                    Navy     14             1   
                                             12             1   
                                             22             1   
                                    Tan      12             1   
                                    Total                   8   
                         MB139      Black    10             1   
                                             12             1   
                                    Tan      12             1   
                                    Burgundy 16             1   
                                             12             1   
                                    Red      10             1   
                                             12             1   
                                             8              2   
                                    Total                   9   
                         awl-1201   Black    10             1   
                                    Blue     14             1   
                                             18             1   
                                             12             2   
                                             22             1   
                                    Total                   6   
                         awl-284    Navy     14             1   
                                             12             1   
                                    Tan      14             1   
                                             12             1   
                                    Oxblood  14             2   
                                    Taupe    14             1   
                                             16             1   
                                             8              1   
                                    Total                   9   
                         awl-blazer Black    14             1   
                                             18             1   
                                    Tan      10             1   
                                             12             1   
                                    Taupe    12             1   
                                    Total                   5   
                         awl-n30    Black    22             1   
                                    Tan      14             1   
                                             16             1   
                                             10             2   
                                             12             2   
                                    Total                   7   
Total                                                      53   

                                                   Total Revenue (£)  \
Category                 Model      Colour   Size                      
Ladies > Leather Jackets 2810       Black    14                  207   
                                             16                   69   
                                             18                  138   
                                    Coffee   10                  138   
                                             12                   69   
                                    

In [355]:
# df = df[(df['customs_description'] == 'Ladies > Leather Jackets')]
# df = df[df['model'] != '5 pads set']

# temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
# temp_refunded = df[df['order_state'] == 'Order Refunded']

# df['model'] = df['model'].fillna("(No assigned category)")
# df['colour'] = df['colour'].fillna("(No assigned colour)")
# df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

# temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
# temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

# temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
# temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

# temp_ladies = pd.concat([temp_dispatched, temp_refunded], axis=1)
# temp_ladies = temp_ladies[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
# temp_ladies['Units Sold'] = temp_ladies['Units Sold'].fillna(0)
# temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].fillna(0)
# temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].fillna(0)
# temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].fillna(0)
# temp_ladies['Final Revenue (£)'] = temp_ladies['Total Revenue (£)'] - temp_ladies['Total Refund (£)']
# temp_ladies.loc['Total', :] = temp_ladies.sum(numeric_only=True).values
# temp_ladies['Units Sold'] = temp_ladies['Units Sold'].astype(int)
# temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].astype(int)
# temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].astype(int)
# temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].astype(int)
# temp_ladies['Final Revenue (£)'] = temp_ladies['Final Revenue (£)'].astype(int)

In [356]:
df = orig_df[(orig_df['date'] >= d1) & (orig_df['date'] <= d2)]
df = df[df['Channel'] == channel]

In [357]:
df = df[(df['Category'] == 'Mens > Leather Jackets')]

df['Model'] = df['Model'].fillna("(No assigned category)")
df['Colour'] = df['Colour'].fillna("(No assigned colour)")
df['Size'] = df['Size'].fillna("(No assigned size)")

if channel == 'Leather Company':
    temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_dispatched_2 = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_refunded = df[df['order_state'] == 'Order Refunded']
    temp_refunded_2 = df[df['order_state'] == 'Order Refunded']

elif channel == 'Debenhams':
    temp_fix_1_df = df.groupby(["order_id", "title"]).filter(lambda x: len(x['order_state'].unique()) == 1)
    temp_fix_2_df = df.groupby(["order_id", "title"]).filter(lambda x: len(x['order_state'].unique()) > 1)
    temp_fix_2_df = temp_fix_2_df[temp_fix_2_df['order_state'] == 'Order Dispatched']
    df = pd.concat([temp_fix_1_df, temp_fix_2_df], ignore_index=True)
    temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_dispatched_2 = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
    temp_refunded = df[df['order_state'] == 'Order Refunded']
    temp_refunded_2 = df[df['order_state'] == 'Order Refunded']

temp_dispatched = temp_dispatched[['Category', 'Model', 'Colour', 'Size', 'quantity']]
temp_dispatched = temp_dispatched.groupby(['Category', 'Model', 'Colour', 'Size'])['quantity'].sum()
temp_dispatched = temp_dispatched.unstack(level=['Colour', 'Size'])
temp_dispatched = temp_dispatched.assign(Total=temp_dispatched.sum(1, min_count=2))
temp_dispatched = temp_dispatched.stack(level=['Colour', 'Size'], sort=False)
# temp_dispatched = temp_dispatched.unstack(level=['Size'])
# temp_dispatched = temp_dispatched.assign(Total=temp_dispatched.sum(1, min_count=2))
# temp_dispatched = temp_dispatched.stack(level=['Size'])
temp_dispatched = temp_dispatched.to_frame(name='quantity')
temp_dispatched.rename(columns={'quantity': 'Units Sold'}, inplace=True)

temp_dispatched_2 = temp_dispatched_2[['Category', 'Model', 'Colour', 'Size', 'price_inc']]
temp_dispatched_2 = temp_dispatched_2.groupby(['Category', 'Model', 'Colour', 'Size'])['price_inc'].sum()
temp_dispatched_2 = temp_dispatched_2.unstack(level=['Colour', 'Size'])
temp_dispatched_2 = temp_dispatched_2.assign(Total=temp_dispatched_2.sum(1, min_count=2))
temp_dispatched_2 = temp_dispatched_2.stack(level=['Colour', 'Size'], sort=False)
# temp_dispatched_2 = temp_dispatched_2.unstack(level=['Size'])
# temp_dispatched_2 = temp_dispatched_2.assign(Total=temp_dispatched_2.sum(1, min_count=2))
# temp_dispatched_2 = temp_dispatched_2.stack(level=['Size'])
temp_dispatched_2 = temp_dispatched_2.to_frame(name='price_inc')
temp_dispatched_2.rename(columns={'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_dispatched = pd.concat([temp_dispatched, temp_dispatched_2], axis=1)



temp_refunded = temp_refunded[['Category', 'Model', 'Colour', 'Size', 'quantity']]
temp_refunded = temp_refunded.groupby(['Category', 'Model', 'Colour', 'Size'])['quantity'].sum()
temp_refunded = temp_refunded.unstack(level=['Colour', 'Size'])
temp_refunded = temp_refunded.assign(Total=temp_refunded.sum(1, min_count=2))
temp_refunded = temp_refunded.stack(level=['Colour', 'Size'], sort=False)
# temp_refunded = temp_refunded.unstack(level=['Size'])
# temp_refunded = temp_refunded.assign(Total=temp_refunded.sum(1, min_count=2))
# temp_refunded = temp_refunded.stack(level=['Size'])
temp_refunded = temp_refunded.to_frame(name='quantity')
temp_refunded.rename(columns={'quantity': 'Units Refunded'}, inplace=True)

temp_refunded_2 = temp_refunded_2[['Category', 'Model', 'Colour', 'Size', 'price_inc']]
temp_refunded_2 = temp_refunded_2.groupby(['Category', 'Model', 'Colour', 'Size'])['price_inc'].sum()
temp_refunded_2 = temp_refunded_2.unstack(level=['Colour', 'Size'])
temp_refunded_2 = temp_refunded_2.assign(Total=temp_refunded_2.sum(1, min_count=2))
temp_refunded_2 = temp_refunded_2.stack(level=['Colour', 'Size'], sort=False)
# temp_refunded_2 = temp_refunded_2.unstack(level=['Size'])
# temp_refunded_2 = temp_refunded_2.assign(Total=temp_refunded_2.sum(1, min_count=2))
# temp_refunded_2 = temp_refunded_2.stack(level=['Size'])
temp_refunded_2 = temp_refunded_2.to_frame(name='price_inc')
temp_refunded_2.rename(columns={'price_inc': 'Total Refund (£)'}, inplace=True)

temp_refunded = pd.concat([temp_refunded, temp_refunded_2], axis=1)

temp = pd.concat([temp_dispatched, temp_refunded], axis=1)

temp = temp[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp['Units Sold'] = temp['Units Sold'].fillna(0)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].fillna(0)
temp['Units Refunded'] = temp['Units Refunded'].fillna(0)
temp['Total Refund (£)'] = temp['Total Refund (£)'].fillna(0)

df_total_sold = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
df_total_refund = df[df['order_state'] == 'Order Refunded']

total_list = [df_total_sold['quantity'].sum(), df_total_sold['price_inc'].sum(),
            df_total_refund['quantity'].sum(), df_total_refund['price_inc'].sum()]
temp.loc['Total', :] = total_list

temp['Final Revenue (£)'] = temp['Total Revenue (£)'] - temp['Total Refund (£)']
temp['Units Sold'] = temp['Units Sold'].astype(int)
temp['Units Refunded'] = temp['Units Refunded'].astype(int)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].astype(int)
temp['Total Refund (£)'] = temp['Total Refund (£)'].astype(int)
temp['Final Revenue (£)'] = temp['Final Revenue (£)'].astype(int)

In [358]:
# df = df[(df['customs_description'] == 'Mens > Leather Jackets')]

# temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
# temp_refunded = df[df['order_state'] == 'Order Refunded']

# df['model'] = df['model'].fillna("(No assigned category)")
# df['colour'] = df['colour'].fillna("(No assigned colour)")
# df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

# temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
# temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

# temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
# temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

# temp = pd.concat([temp_dispatched, temp_refunded], axis=1)
# temp = temp[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
# temp['Units Sold'] = temp['Units Sold'].fillna(0)
# temp['Total Revenue (£)'] = temp['Total Revenue (£)'].fillna(0)
# temp['Units Refunded'] = temp['Units Refunded'].fillna(0)
# temp['Total Refund (£)'] = temp['Total Refund (£)'].fillna(0)
# temp['Final Revenue (£)'] = temp['Total Revenue (£)'] - temp['Total Refund (£)']
# temp.loc['Total', :] = temp.sum(numeric_only=True).values
# temp['Units Sold'] = temp['Units Sold'].astype(int)
# temp['Units Refunded'] = temp['Units Refunded'].astype(int)
# temp['Total Revenue (£)'] = temp['Total Revenue (£)'].astype(int)
# temp['Total Refund (£)'] = temp['Total Refund (£)'].astype(int)
# temp['Final Revenue (£)'] = temp['Final Revenue (£)'].astype(int)

In [359]:
final = pd.concat([temp_ladies, temp])
# final.index=pd.MultiIndex.from_tuples([(x[0], x[1], x[2].replace('ZTotal', 'Total')) for x in final.index])
# final = final.unstack(level=['Colour', 'Size'])
# final = final.index.str.replace('ZTotal', 'Total')
# final = final.stack(level=['Colour', 'Size'])

In [360]:
final

Units Sold  \
Category                 Model      Colour   Size               
Ladies > Leather Jackets 2810       Black    14             3   
                                             16             1   
                                             18             2   
                                    Coffee   10             2   
                                             12             1   
                                    Total                   9   
                         AWL-281    Black    14             1   
                                             18             1   
                                    Burgandy 22             1   
                                    Grey     12             1   
                                    Navy     14             1   
                                             12             1   
                                             22             1   
                                    Tan      12             1   
                                    Total                   8   
                         MB139      Black    10             1   
                                             12             1   
                                    Tan      12             1   
                                    Burgundy 16             1   
                                             12             1   
                                    Red      10             1   
                                             12             1   
                                             8              2   
                                    Total                   9   
                         awl-1201   Black    10             1   
                                    Blue     14             1   
                                             18             1   
                                             12             2   
                                             22             1   
                                    Total                   6   
                         awl-284    Navy     14             1   
                                             12             1   
                                    Tan      14             1   
                                             12             1   
                                    Oxblood  14             2   
                                    Taupe    14             1   
                                             16             1   
                                             8              1   
                                    Total                   9   
                         awl-blazer Black    14             1   
                                             18             1   
                                    Tan      10             1   
                                             12             1   
                                    Taupe    12             1   
                                    Total                   5   
                         awl-n30    Black    22             1   
                                    Tan      14             1   
                                             16             1   
                                             10             2   
                                             12             2   
                                    Total                   7   
Total                                                      53   
                                                            0   

                                                   Total Revenue (£)  \
Category                 Model      Colour   Size                      
Ladies > Leather Jackets 2810       Black    14                  207   
                                             16                   69   
                                             18                  138   
                                    Coffee   10                  138   
                                           

In [361]:
final.to_excel(f'{channel} - {month_abb} 2024 - Leather Jackets.xlsx')

In [362]:
# # Leather Company
# import matplotlib.pyplot as plt

# temp_dispatched = temp_dispatched[temp_dispatched['price_inc'] < 400]
# temp_refunded = temp_refunded[temp_refunded['price_inc'] < 400]

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
# fig.suptitle('Leather Company - Ladies Leather Jackets')
# plt.figure(figsize=(20,6))
# bins = [109, 119, 129, 139, 149, 159, 169, 179]
# ax1.hist(temp_dispatched['price_inc'], bins=10, edgecolor="k")
# ax1.set_title('Sold')
# ax2.hist(temp_refunded['price_inc'], bins=10, edgecolor="k")
# ax2.set_title('Refunded')
# y_bins = [0, 10, 20, 30, 40, 50]
# ax1.set_yticks(y_bins)
# ax2.set_yticks(y_bins)
# # ax1.set_xticks(bins)
# # ax2.set_xticks(bins)
# plt.show()